# Example parameter sweeps


In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import scqubits as scq
from scqubits import HilbertSpace, InteractionTerm, ParameterSweep

import numpy as np

In [8]:
qubit = scq.GenericQubit(E = 4.0)
oscillator = scq.Oscillator(
    E_osc=4.5,
    truncated_dim=4    # up to 3 photons (0,1,2,3)
)


In [7]:
fluxonium = scq.Fluxonium.create()

Output()

In [2]:
# Define HilbertSpace object:  two transmons coupled to an oscillator

tmon1 = scq.TunableTransmon(
    EJmax=40.0,
    EC=0.2,
    d=0.1,
    flux=0.23,
    ng=0.3,
    ncut=40,
    truncated_dim=3,     # after diagonalization, we will keep 3 levels
    id_str="tmon1"       # optional; used for referencing the object from within ParameterSweep or HilbertSpace
)

tmon2 = scq.TunableTransmon(
    EJmax=15.0,
    EC=0.15,
    d=0.2,
    flux=0.0,
    ng=0.0,
    ncut=30,
    truncated_dim=3,
    id_str="tmon2"
)

resonator = scq.Oscillator(
    E_osc=4.5,
    truncated_dim=4    # up to 3 photons (0,1,2,3)
)

hilbertspace = scq.HilbertSpace([tmon1, tmon2, resonator])


g1 = 0.1  # coupling resonator-CPB1 (without charge matrix elements)
g2 = 0.2  # coupling resonator-CPB2 (without charge matrix elements)

hilbertspace.add_interaction(
    g_strength = g1,
    op1 = tmon1.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True,
    id_str="tmon1-resonator"  # optional keyword argument
)

hilbertspace.add_interaction(
    g_strength = g2,
    op1 = tmon2.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True,
    id_str="tmon2-resonator"  # optional keyword argument
)

In [19]:
hilbertspace2 = scq.HilbertSpace([qubit, oscillator])
hilbertspace2.add_interaction(
    g_strength = g1,
    op1 = qubit.sm_operator,
    op2 = oscillator.creation_operator,
    add_hc = True,
    id_str="JCint"
)

In [16]:
hilbertspace3 = scq.HilbertSpace([fluxonium])

In [17]:
# Set up parameter name and values
pname1 = 'flux'  
flux_vals = np.linspace(0.0, 2.0, 11)
pname2 = 'ng'
ng_vals = np.linspace(-0.5, 0.5, 9)

# combine into a dictionary
paramvals_by_name = {pname1: flux_vals, pname2: ng_vals}


area_ratio = 1.2

def update_hilbertspace(flux, ng):  # function that defines how Hilbert space components are updated
    tmon1.flux = flux
    tmon2.flux = area_ratio * flux
    tmon2.ng = ng

# dictionary with information on which subsystems are affected by changing parameters    
subsys_update_info = {pname1: [tmon1, tmon2],
                      pname2: [tmon2]}

    
# create the ParameterSweep    
sweep = ParameterSweep(
    hilbertspace=hilbertspace,
    paramvals_by_name=paramvals_by_name,
    update_hilbertspace=update_hilbertspace,
    evals_count=20,
    subsys_update_info=subsys_update_info,
    num_cpus=4
)

Parallel compute bare eigensys [num_cpus=4]

Parallel compute bare eigensys [num_cpus=4]

Parallel compute bare eigensys [num_cpus=4]

Parallel compute dressed eigensys [num_cpus=4]

In [26]:
pname3 = 'coupling'
g_vals = np.linspace(2.0, 7.0, 40)


paramvals_by_name2 = {pname3: g_vals}

def update_hilbertspace2(g):
    hilbertspace2["JCint"].g_strength = g

sweep2 = ParameterSweep(
    hilbertspace=hilbertspace2,
    paramvals_by_name=paramvals_by_name2,
    update_hilbertspace=update_hilbertspace2,
    evals_count=4,
    num_cpus=4
)

Parallel compute bare eigensys [num_cpus=4]

Parallel compute bare eigensys [num_cpus=4]

Parallel compute dressed eigensys [num_cpus=4]

In [27]:
pname4 = 'EL'
EL_vals = np.linspace(0.001, 1.0, 40)

paramvals_by_name3 = {pname4: EL_vals}

def update_hilbertspace3(EL):
    hilbertspace3[0].EL = EL

sweep3 = ParameterSweep(
    hilbertspace=hilbertspace3,
    paramvals_by_name=paramvals_by_name3,
    update_hilbertspace=update_hilbertspace3,
    evals_count=6,
    num_cpus=4
)

Parallel compute bare eigensys [num_cpus=4]

Parallel compute dressed eigensys [num_cpus=4]